<a href="https://colab.research.google.com/github/iwatake2222/colaboratory_study/blob/master/object_classification/cat_and_dog/mobilenet_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MobileNet ファインチューニング

学習用画像 (Cat and Dog): 
https://drive.google.com/drive/u/0/folders/139S7L2uDqgHxtVeV6lzjCg57ZJzybZCJ

参考: https://colab.research.google.com/github/tensorflow/docs/blob/master/site/ja/tutorials/load_data/images.ipynb


## 環境設定

In [0]:
from google.colab import drive 
drive.mount("/content/drive")
!ls

## パラメータ

In [0]:
MODEL_WIDTH = 224
MODEL_HEIGHT = 224
BATCH_SIZE = 32
RATIO_VALIDATION = 0.8

## データセットのダウンロード

In [0]:
%%bash
FILE_ID=1-9TGQITGsysMFhY5X5cJw2Img3FL1h85
FILE_NAME=dataset_cat_and_dog.tar.gz

curl -sc /tmp/cookie "https://drive.google.com/uc?export=download&id=${FILE_ID}" > /dev/null
CODE="$(awk '/_warning_/ {print $NF}' /tmp/cookie)"  
curl -Lb /tmp/cookie "https://drive.google.com/uc?export=download&confirm=${CODE}&id=${FILE_ID}" -o ${FILE_NAME}
tar -xvf ${FILE_NAME}

## データセットの作成

In [0]:
%tensorflow_version 2.x
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

import pathlib
import random

# 全画像を取得
data_root = pathlib.Path("dataset_cat_and_dog")
all_image_paths = list(data_root.glob('*/*'))
all_image_paths = [str(path) for path in all_image_paths]
random.shuffle(all_image_paths)


## 画像をチェック
import cv2
from google.colab.patches import cv2_imshow
for i in range(3):
  image_path = random.choice(all_image_paths)
  img_org = cv2.imread(image_path)
  cv2_imshow(img_org)

# ラベルを取得
label_names = sorted(item.name for item in data_root.glob('*/') if item.is_dir())
label_to_index = dict((name, index) for index,name in enumerate(label_names))
all_image_labels = [label_to_index[pathlib.Path(path).parent.name] for path in all_image_paths]

# 学習用とテスト用に分ける
num_image = len(all_image_paths)
num_train_image = (int)(num_image * RATIO_VALIDATION)
num_test_image = num_image - num_train_image

train_image_paths = all_image_paths[:-num_test_image]
test_image_paths = all_image_paths[-num_test_image:]
train_image_labels = all_image_labels[:-num_test_image]
test_image_labels = all_image_labels[-num_test_image:]

In [0]:
# データローダ、Preprocess関数
def preprocess_image(image):
  image = tf.image.resize(image, [MODEL_HEIGHT, MODEL_WIDTH])
  image /= 255.0  # normalize to [0,1] range
  return image

def load_and_preprocess_image(path):
  image = tf.io.read_file(path)
  image = tf.image.decode_jpeg(image, channels=3)
  return preprocess_image(image)

import matplotlib.pyplot as plt
plt.imshow(load_and_preprocess_image(all_image_paths[0]))

In [0]:
# データセット作成
image_ds = tf.data.Dataset.from_tensor_slices(train_image_paths).map(load_and_preprocess_image, tf.data.experimental.AUTOTUNE)
label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(train_image_labels, tf.int32))
train_ds = tf.data.Dataset.zip((image_ds, label_ds))

image_ds = tf.data.Dataset.from_tensor_slices(test_image_paths).map(load_and_preprocess_image, tf.data.experimental.AUTOTUNE)
label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(test_image_labels, tf.int32))
test_ds = tf.data.Dataset.zip((image_ds, label_ds))

## check
# sample = next(iter(train_ds))
# label_names[sample[1].numpy()]
# plt.imshow(sample[0].numpy())

# for d in train_ds.take(10):
#   print(d[1].numpy)
#   plt.imshow(d[0].numpy())
#   plt.show()


# データセットを学習用に設定
train_ds = train_ds.cache()
train_ds = train_ds.shuffle(buffer_size=num_train_image).repeat().batch(BATCH_SIZE)
train_ds = train_ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

test_ds = train_ds.cache()
test_ds = test_ds.shuffle(buffer_size=num_test_image).repeat().batch(BATCH_SIZE)
test_ds = train_ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

# 値範囲を0.0-1.0 -> -1.0-1.1に変更
def change_range(image,label):
  return 2 * image - 1, label

train_ds = train_ds.map(change_range)
test_ds = test_ds.map(change_range)

## モデルの作成と学習

In [0]:
# ベースとなるモデルを読み込む
model_base = tf.keras.applications.MobileNetV2(input_shape=(MODEL_HEIGHT, MODEL_WIDTH, 3), include_top=False)
model_base.trainable = False

# 1/4のレイヤだけ学習する
# for layer in model_base.layers[:-int(len(model_base.layers) / 4)]:
# 	layer.trainable = True

# 出力層を追加する
output_without_fc = model_base.output
avg1 = tf.keras.layers.GlobalAveragePooling2D()(output_without_fc)
fc1 = tf.keras.layers.Dense(units=64, activation=tf.keras.activations.relu)(avg1)
dropout1 = tf.keras.layers.Dropout(0.2)(fc1)
fc2 = tf.keras.layers.Dense(units=len(label_names), activation=tf.keras.activations.softmax)(dropout1)
model = tf.keras.models.Model(inputs=model_base.input, outputs=fc2)

# モデルを作成する
model = tf.keras.models.Model(inputs=model_base.input, outputs=fc2)
model.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
  metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
)
model.summary()

In [0]:
# 学習する
cp_callback = tf.keras.callbacks.ModelCheckpoint("ep_{epoch:02d}.h5", save_best_only=True)
train_steps_per_epoch=tf.math.ceil(num_train_image/BATCH_SIZE).numpy()
validate_steps_per_epoch=tf.math.ceil(num_test_image/BATCH_SIZE).numpy()
model.fit(train_ds, epochs=50, steps_per_epoch=train_steps_per_epoch, validation_data=test_ds, validation_steps=validate_steps_per_epoch, callbacks=[cp_callback])

# 簡易評価する
sample = next(iter(test_ds))
model.evaluate(sample[0], sample[1])

## モデルを保存する

In [0]:
model.save("mobilenet_v2_dogcat.h5")

# Save to google drive
# !cp conv_mnist.h5  "/content/drive/My Drive/"

# Download to local
from google.colab import files
files.download("./mobilenet_v2_dogcat.h5")

## 推論テスト

In [0]:
# Set test image
# img_test_path = "dataset_cat_and_dog/cat/cat9302331.jpg"
uploaded = files.upload()
img_test_path = next(iter(uploaded.keys()))

%tensorflow_version 2.x
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import numpy as np
import cv2
from google.colab.patches import cv2_imshow


# Load model
loaded_model = tf.keras.models.load_model("mobilenet_v2_dogcat.h5")
# loaded_model.summary()
model_n, model_h, model_w, model_c = loaded_model.input_shape

# Read image
img_org = cv2.imread(img_test_path)
cv2_imshow(img_org)

# Pre process
img = cv2.resize(img_org, (model_w, model_h))
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
input_tensor = img / 128.
input_tensor -= 1.
input_tensor = input_tensor.astype(np.float32)
input_tensor = np.expand_dims(input_tensor, axis=0)

# Inference
output_tensor = loaded_model(inputs=input_tensor)

# Post process
result = np.argmax(output_tensor)
score = output_tensor[0][result].numpy()
print("predicted number is {} [{:.2f}]".format(result, score))